In [1]:
#Results were produced in stints. This is the number of the last stint.
run = 3

In [2]:
import numpy as np
import tensorflow as tf
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from scipy import stats
import joblib

#folder for saving results
filepath = ".../Resultate_final/Put/saved_sim/"

In [3]:
#Market and option parameters as in section 4.1 of 'Assessing Asset-Liability Risk with Neural Networks' (Cheridito, Ery, Wüthrich 2020)
s_0 = 100
r = 0.01
mu = 0.05
sigma = 0.2
tau = 1/52
T = 1/3
K = 100

#Confidence levels and parameters for Value-at-Risk, Expected Shortfall and GlueVaR
alpha_VaR = 0.995
alpha_ES = 0.99
alpha_Glue = 0.95
beta_Glue = 0.995
omega_Glue = np.array([1/3,1/3])

In [4]:
#Sizes for training set, validation set, test set, and set size for Monte Carlo estimation of the risk measures
M_1 = 1500000
M_2 = 500000
M_3 = 500000
M_MC = 500000

In [5]:
#Function for calculating simulated values of S_tau and simulated payoffs P_T from simulations of standard normal random variables
def data_gen(Z,V):
    #simulate S_tau under P
    S_tau = s_0 * np.exp( (mu-0.5*sigma**2)*tau + sigma*np.sqrt(tau)*Z)
    #Simulate S_T given S_tau under Q
    S_T = S_tau * np.exp( (r-0.5*sigma**2)*(T-tau) + sigma*np.sqrt(T-tau)*V)
    #Calculate corresponding simulated discounted payoffs
    P_T = np.exp(-r*(T-tau)) * np.maximum(K-S_T,0)
    return S_tau, P_T

#Function for the computation of GlueVaR
def GlueVaR(omega, L, alpha, beta):
    j_beta = int(len(L)*(1-beta))-1
    j_alpha = int(len(L)*(1-alpha))-1

    ES_beta = 1/(1-beta) * np.sum(L[0:j_beta-1])/len(L) + ( 1 - (j_beta-1)/((1-beta)*len(L)) )*L[j_beta]
    ES_alpha = 1/(1-alpha) * np.sum(L[0:j_alpha-1])/len(L) + ( 1 - (j_alpha-1)/((1-alpha)*len(L)) )*L[j_alpha]
    VaR_alpha = L[j_alpha]

    return omega[0]*ES_beta + omega[1]*ES_alpha + (1-omega[0]-omega[1])*VaR_alpha

In [6]:
for j in range(17):
    #Generating simulations of standard normal random variables for training set, validation set, test set, set for Monte Carlo estimation of risk measures
    Z_train = np.random.normal(loc=0, scale=1, size=M_1)
    V_train = np.random.normal(loc=0, scale=1, size=M_1)
    Z_val = np.random.normal(loc=0, scale=1, size=M_2)
    V_val = np.random.normal(loc=0, scale=1, size=M_2)
    Z_test = np.random.normal(loc=0, scale=1, size=M_3)
    V_test = np.random.normal(loc=0, scale=1, size=M_3)
    Z_MC = np.random.normal(loc=0, scale=1, size=M_MC)
    V_MC = np.random.normal(loc=0, scale=1, size=M_MC)

    #Calculate the risk factor S_tau and the corresponding simulated payoffs P_T
    S_tau_train, P_T_train = data_gen(Z=Z_train, V=V_train)
    S_tau_val, P_T_val = data_gen(Z=Z_val, V=V_val)
    S_tau_test, P_T_test = data_gen(Z=Z_test, V=V_test)
    S_tau_MC, P_T_MC = data_gen(Z=Z_MC, V=V_MC)

    #define and compile neural network model, setup as in paper
    bi = np.log( np.sum(P_T_train)/len(P_T_train))
    model = tf.keras.models.Sequential([
        tf.keras.layers.BatchNormalization(input_shape=(1,)),
        tf.keras.layers.Dense(5, activation='tanh'),
        tf.keras.layers.Dense(1, activation='exponential', bias_initializer=tf.keras.initializers.Constant(value=bi))])
    model.compile(loss='mse', optimizer='adam', metrics=['mse'])
    #training of the neural network
    hist = model.fit(x=S_tau_train, y=P_T_train, epochs=40, batch_size=10000, validation_data=(S_tau_val,P_T_val), verbose=0)

    #training of the random forest, hyperparameters taken from Put.ipynb
    rfr = RandomForestRegressor(n_estimators=400, criterion='squared_error', min_samples_leaf=2000, bootstrap=True, verbose=0, warm_start=True, n_jobs=-1)
    rfr.fit(X=S_tau_train.reshape(-1,1), y=P_T_train)


    #Calculating the parts of the test set that fall into B_1 and B_2
    s_40 = s_0 * np.exp( (mu-0.5*sigma**2)*tau + sigma*np.sqrt(tau)*stats.norm.ppf(0.4, loc=0, scale=1))
    s_70 = s_0 * np.exp( (mu-0.5*sigma**2)*tau + sigma*np.sqrt(tau)*stats.norm.ppf(0.7, loc=0, scale=1))
    B_1 = S_tau_test < s_40
    B_2 = S_tau_test > s_70

    #computation of the metrics (a), (b), (c) with B_1 and (c) with B_2 for the neural network
    P_T_pred_NN = model.predict(S_tau_test)[:,0]
    mse_train_NN = hist.history['mse'][-1]
    mse_val_NN = hist.history['val_mse'][-1]
    mc_tmp = P_T_pred_NN - P_T_test
    metric_a_NN = np.sum(mc_tmp)/len(P_T_test)
    metric_b_NN = np.sum((mc_tmp)*P_T_pred_NN)/len(P_T_test)
    metric_c_B_1_NN = np.sum(mc_tmp[B_1])/len(P_T_test)
    metric_c_B_2_NN = np.sum(mc_tmp[B_2])/len(P_T_test)

    #computation of the metrics (a), (b), (c) with B_1 and (c) with B_2 for the random forest
    mse_train_RF = mean_squared_error(y_pred=rfr.predict(S_tau_train.reshape(-1,1)),y_true=P_T_train)
    P_T_pred_RF = rfr.predict(S_tau_test.reshape(-1,1))
    mse_val_RF = mean_squared_error(y_pred=P_T_pred_RF,y_true=P_T_test)
    mc_tmp = P_T_pred_RF - P_T_test
    metric_a_RF = np.sum(mc_tmp)/len(P_T_test)
    metric_b_RF = np.sum((mc_tmp)*P_T_pred_RF)/len(P_T_test)
    metric_c_B_1_RF = np.sum(mc_tmp[B_1])/len(P_T_test)
    metric_c_B_2_RF = np.sum(mc_tmp[B_2])/len(P_T_test)

    #calulation of risk measures
    #calculating realisations of \widehat{L}, here denoted by P_T_pred
    P_T_pred_MC_NN = model.predict(S_tau_MC)[:,0]
    P_T_pred_MC_RF = rfr.predict(S_tau_MC.reshape(-1,1))

    #sorting observations
    L_NN = np.sort(P_T_pred_MC_NN)[::-1]
    L_RF = np.sort(P_T_pred_MC_RF)[::-1]

    #Value-at-Risk
    j_VaR = int(M_MC*(1-alpha_VaR))-1
    VaR_hat_NN = L_NN[j_VaR]
    VaR_hat_RF = L_RF[j_VaR]

    #Expected Shortfall
    j_ES = int(M_MC*(1-alpha_ES))-1
    ES_hat_NN = 1/(1-alpha_ES) * np.sum(L_NN[0:j_ES-1])/M_MC + ( 1 - (j_ES-1)/((1-alpha_ES)*M_MC) )*L_NN[j_ES]
    ES_hat_RF = 1/(1-alpha_ES) * np.sum(L_RF[0:j_ES-1])/M_MC + ( 1 - (j_ES-1)/((1-alpha_ES)*M_MC) )*L_RF[j_ES]
    
    #GlueVaR
    GlueVaR_hat_NN = GlueVaR(omega=omega_Glue, L=L_NN, alpha=alpha_Glue, beta=beta_Glue)
    GlueVaR_hat_RF = GlueVaR(omega=omega_Glue, L=L_RF, alpha=alpha_Glue, beta=beta_Glue)

    #save results for further evaluation
    output = np.array([[mse_train_NN,mse_val_NN,metric_a_NN,metric_b_NN,metric_c_B_1_NN,metric_c_B_2_NN,VaR_hat_NN,ES_hat_NN,GlueVaR_hat_NN],
                      [mse_train_RF,mse_val_RF,metric_a_RF,metric_b_RF,metric_c_B_1_RF,metric_c_B_2_RF,VaR_hat_RF,ES_hat_RF,GlueVaR_hat_RF]])

    joblib.dump(output,filepath+'output_'+str(run)+'_'+str(j)+'.joblib')
    #prints just for checking while the notebook is running
    print(j)
    print(mse_val_NN,mse_val_RF)

2021-11-09 18:22:38.052858: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-11-09 18:22:40.177573: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 30988 MB memory:  -> device: 0, name: Tesla V100-SXM2-32GB, pci bus id: 0000:16:00.0, compute capability: 7.0
2021-11-09 18:22:40.179192: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 30988 MB memory:  -> device: 1, name: Tesla V100-SXM2-32GB, pci bus id: 0000:3a:00.0, compute capability: 7.0
2021-11-09 18:22:40.180732: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:

0
36.49256896972656 36.36761807049517
1
36.30271911621094 36.28382218138532
2
36.44670867919922 36.21964702118515
3
36.330772399902344 36.38972251224388
4
36.36061477661133 36.29348043378203
5
36.4785270690918 36.419567332838916
6
36.395565032958984 36.47652487810508
7
36.2723274230957 36.52228017725057
8
36.422969818115234 36.45520438158813
9
36.442474365234375 36.50504095241592
10
36.46348190307617 36.36702382549602
11
36.43077850341797 36.30834518299279
12
36.37877655029297 36.385144154775126
13
36.523704528808594 36.52693616048291
14
36.29164505004883 36.48368433867632
15
36.32743453979492 36.47279997999922
16
36.29365921020508 36.39300426220398
